In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

%load_ext autoreload
%autoreload 2

from CrossLS import CrossLS
import pymatching
import sinter
from typing import List
import pandas as pd
import numpy as np
# from bposdd import BPOSD
from stimbposd import BPOSD
from stimbposd import sinter_decoders


In [2]:
# Compute the logical error rate (LER) per round
def convert_to_per_round(total_LER: float, rounds: int) -> float:
    per_round_LER = 1/2 * (1 - np.abs(1-2*total_LER)**(1/rounds))
    return per_round_LER

### 1. Regular BPOSD

We use a small size example to show the bias in LER when teleporting different states on the PQRM code.

In [3]:
rx = 1
rz = 2
m = 4
d_surf = 3
rounds = d_surf
noise = 0.001
noise_profile = [1e-6, noise, noise, noise]

In [4]:
tasks = []
for PQRM_state in ["Z", "X"]: # can also try: for d_surf in [3, 5]
    circuit_124_3 = CrossLS([rx, rz, m], d_surf, rounds, PQRM_state, noise_profile, punctured=True)
    tasks.append(sinter.Task(circuit=circuit_124_3, json_metadata={'d_surf': d_surf, 'PQRM_para': [rx,rz,m], 'rounds': rounds, 'PQRM_state': PQRM_state, 'p1': noise_profile[0], 'p2': noise_profile[1], 'pM': noise_profile[2], 'pR': noise_profile[3]}))

collected_stats: List[sinter.TaskStats] = sinter.collect(
    num_workers=6,
    tasks=tasks,
    decoders=['bposd'],
    max_shots=1000000, # change to large number for full scale experiment
    max_errors=100, # Whichever first hits the threshold stops the simulation
    custom_decoders=sinter_decoders(),
    # custom_decoders={'bposd': BPOSD(max_iter = 1000, bp_method = 'ms', osd_order = 10, osd_method='osd_cs')},
    print_progress=True,
)

# Convert collected_stats into Dataframe
records = []
for stat in collected_stats:
    record = {
        'decoder': stat.decoder,
        'shots': stat.shots,
        'errors': stat.errors,
        'seconds': stat.seconds,
        'LER': convert_to_per_round(stat.errors / stat.shots, 1) if stat.shots > 0 else None, # don't average
        'rounds': stat.json_metadata['rounds'],
        'd_surf': stat.json_metadata['d_surf'],
        'PQRM_para': stat.json_metadata['PQRM_para'],
        'PQRM_state': stat.json_metadata['PQRM_state'],
        'p1': stat.json_metadata['p1'], 'p2': stat.json_metadata['p2'], 'pM': stat.json_metadata['pM'], 'pR': stat.json_metadata['pR']
    }
    records.append(record)
df_sf = pd.DataFrame(records)
df_sf
# df_sf.to_csv("results.csv", index=False) # If you wanna save the experiment data to csv

Starting 6 workers...
2 tasks left:
  workers decoder eta shots_left errors_left json_metadata                                                                         
        3   bposd   ?    1000000         100 d_surf=3,PQRM_para=[1, 2, 4],rounds=3,PQRM_state=Z,p1=1e-06,p2=0.001,pM=0.001,pR=0.001
        3   bposd 11m     999983         100 d_surf=3,PQRM_para=[1, 2, 4],rounds=3,PQRM_state=X,p1=1e-06,p2=0.001,pM=0.001,pR=0.001
2 tasks left:
  workers decoder eta shots_left errors_left json_metadata                                                                         
        3   bposd 12m     999966         100 d_surf=3,PQRM_para=[1, 2, 4],rounds=3,PQRM_state=Z,p1=1e-06,p2=0.001,pM=0.001,pR=0.001
        3   bposd 11m     999983         100 d_surf=3,PQRM_para=[1, 2, 4],rounds=3,PQRM_state=X,p1=1e-06,p2=0.001,pM=0.001,pR=0.001
2 tasks left:
  workers decoder eta shots_left errors_left json_metadata                                                                         
        3   

,decoder,shots,errors,seconds,LER,rounds,d_surf,PQRM_para,PQRM_state,p1,p2,pM,pR
0,bposd,6963,133,10.319972,0.019101,3,3,"[1, 2, 4]",X,0.000001,0.001,0.001,0.001
1,bposd,158072,123,220.372377,0.000778,3,3,"[1, 2, 4]",Z,0.000001,0.001,0.001,0.001


We can see that teleporting $|0\rangle$ using ZZ measurement has lower LER than teleporting $|+\rangle$ state, and only the $|0\rangle$ case gives the desired LER suppression as the $d_{surf}$ increases (for the $|+\rangle$ case the LER remain at 0.01 level). To successfully teleporting a non-Clifford gate, we need to fix this bias.

### 2. Post-selection on perfect X-stabilizer syndrome

In [6]:
# Post-selection
num_shots = 100000
PQRM_state = "X"
circuit_124_3 = CrossLS([rx, rz, m], d_surf, rounds, PQRM_state, noise_profile, punctured=True)
sampler = circuit_124_3.compile_detector_sampler()
shots, observables = sampler.sample(num_shots, separate_observables=True)

# LER without decoding
LER_no_decode = np.sum(observables, axis=0)/num_shots
print("LER without decoding: ", convert_to_per_round(LER_no_decode, 1))

postselect_indices = np.all(shots[:, -m:] == 0, axis=1)
shots_post = shots[postselect_indices, :]
observable_post = observables[postselect_indices, :]
print(f"Post-selection rate: {shots_post.shape[0]/num_shots}")

decoder = BPOSD(circuit_124_3.detector_error_model(), max_bp_iters=500)
predicted_observables_post = decoder.decode_batch(shots_post)
num_mistakes = np.sum(np.any(predicted_observables_post != observable_post, axis=1))

print(f"LER: {num_mistakes/shots_post.shape[0]}")


LER without decoding:  [0.03183]
Post-selection rate: 0.87541
LER: 0.00031985012736889


We can see that the LER drops to similar level as the $|0\rangle$ case.

In [ ]:
# Joint decoding: first decode RM measurement results using classical RM decoder
# then put back, construct new detector values
# num_shots = 10
# sampler = circuit_124_3.compile_sampler() # return measurement results
# measurement_results = sampler.sample(num_shots)

# # correct the measurement result on PQRM data qubits
# # 2 replace by d_PQRM-1
# d_PQRM = 3 # 3 or 7
# measurement_results_int = measurement_results.astype(int)
# for y in range(0, d_PQRM-1):
#     data = (-1, 3 + 2*y)
#     flip_data = [(0,2+2*y0) for y0 in range(y+1)]
#     flip_data_indices = [-final_data1[::-1].index(coord) -1 - len(final_data2) for coord in flip_data]
#     base_col_index = -len(final_data2)-2+y
#     measurement_results_int[:, flip_data_indices] += measurement_results_int[:, [base_col_index]]
#     measurement_results_int = measurement_results_int % 2

# # Classical RM decoder
# noisy_cws= measurement_results_int[:, -len(final_data1+final_data2):-len(final_data2)-(d_PQRM - 1)]
# decoded_cws = RM_decoder(rz,m, noisy_cws)
# print(np.sum(decoded_cws, axis=1))
# measurement_results_int[:, -len(final_data1+final_data2):-len(final_data2)-(d_PQRM - 1)] = decoded_cws
# measurement_results = measurement_results_int.astype(bool)
# converter = circuit_124_3.compile_m2d_converter()
# detector_fixed, observables_fixed = converter.convert(measurements=measurement_results, separate_observables=True)

# # Feed to BPOSD
# decoder = BPOSD(circuit_124_3.detector_error_model(), max_bp_iters=500)
# predicted_observables_fixed = decoder.decode_batch(detector_fixed)
# num_mistakes = np.sum(np.any(predicted_observables_fixed != observables_fixed, axis=1))

# print(f"LER: {num_mistakes/detector_fixed.shape[0]}")